In [1]:
import pandas as pd
import re
import multiprocessing

In [2]:
cta_train_gt = pd.read_csv('SOTAB CTA/sotab_v2_cta_training_set.csv')
cta_test_gt = pd.read_csv('SOTAB CTA/sample_test.csv')

In [3]:
cta_train_gt

,table_name,column_index,label
0,Book_11x17.pt_September2020_CTA.json.gz,3,Date
1,Book_12min.com_September2020_CTA.json.gz,0,Book/name
2,Book_12min.com_September2020_CTA.json.gz,2,Language
3,Book_1carpetcleaning.co.uk_September2020_CTA.j...,7,Person/name
4,Book_1carpetcleaning.co.uk_September2020_CTA.j...,2,BookFormatType
...,...,...,...
116882,TVEpisode_zazangels.com_September2020_CTA.json.gz,3,CreativeWorkSeries
116883,TVEpisode_zebrahead.org_September2020_CTA.json.gz,0,TVEpisode/name
116884,TVEpisode_zebrahead.org_September2020_CTA.json.gz,3,CreativeWorkSeries
116885,TVEpisode_zoids-col.net_September2020_CTA.json.gz,1,TVEpisode/name


In [4]:
cta_test_gt

,table_name,column_index,label
0,Event_blugrottonj.com_September2020_CTA.json.gz,4,Place/name
1,Event_blugrottonj.com_September2020_CTA.json.gz,5,telephone
2,LocalBusiness_basellive.ch_September2020_CTA.j...,2,openingHours
3,LocalBusiness_worldcement.com_September2020_CT...,5,Country
4,Movie_321movies.org_September2020_CTA.json.gz,10,Rating
...,...,...,...
313,Product_babyartikelcheck.de_September2020_CTA....,0,Product/name
314,Product_divas-club.de_September2020_CTA.json.gz,0,Product/name
315,Product_divas-club.de_September2020_CTA.json.gz,1,Product/description
316,Person_crsdenver.com_September2020_CTA.json.gz,3,faxNumber


In [5]:
cta_test_gt["class"] = cta_test_gt["table_name"].apply(lambda x: x.split("_")[0])

In [6]:
cta_test_gt

,table_name,column_index,label,class
0,Event_blugrottonj.com_September2020_CTA.json.gz,4,Place/name,Event
1,Event_blugrottonj.com_September2020_CTA.json.gz,5,telephone,Event
2,LocalBusiness_basellive.ch_September2020_CTA.j...,2,openingHours,LocalBusiness
3,LocalBusiness_worldcement.com_September2020_CT...,5,Country,LocalBusiness
4,Movie_321movies.org_September2020_CTA.json.gz,10,Rating,Movie
...,...,...,...,...
313,Product_babyartikelcheck.de_September2020_CTA....,0,Product/name,Product
314,Product_divas-club.de_September2020_CTA.json.gz,0,Product/name,Product
315,Product_divas-club.de_September2020_CTA.json.gz,1,Product/description,Product
316,Person_crsdenver.com_September2020_CTA.json.gz,3,faxNumber,Person


In [7]:
gt = {'train':{}, 'test':{}}
for index, row in cta_train_gt.iterrows():
    if row["table_name"] not in gt['train']:
        gt['train'][row["table_name"]] = {}
    gt['train'][row["table_name"]][row["column_index"]] = row["label"]
 
for index, row in cta_test_gt.iterrows():
    if row["table_name"] not in gt['test']:
        gt['test'][row["table_name"]] = {}
    gt['test'][row["table_name"]][row["column_index"]] = row["label"]


In [8]:
gt['test']['Event_blugrottonj.com_September2020_CTA.json.gz']

{4: 'Place/name', 5: 'telephone'}

In [9]:
def clean_text(text):
        
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    
    text = re.sub(' +', ' ', str(text)).strip()
    
    return text

In [ ]:
def get_table(file_name):
    
 
    if file_name in cta_train_gt["table_name"].tolist():
        path = 'SOTAB CTA/Train/'+file_name
        split = "train"
    else:
        path = 'SOTAB CTA/Test/'+file_name
        split = "test"


    df = pd.read_json(path, compression='gzip', lines=True)
    labeled_idx = gt[split][file_name]
    df = df[[0]+list(labeled_idx.keys())]                #[0]+  for cpa
    

    display(df[list(labeled_idx.keys())])
    print(labeled_idx)

    cleaned_rows = []

    for row in gt["test"]:
        col_labels = """"""
        for i, l in enumerate(df[list(labeled_idx.keys())]):
            col_labels += f"""Column {i+1}: {df[list(labeled_idx.keys())] }   \n"""
        cleaned_rows.append(col_labels.strip())

    return cleaned_rows



    # cleaned_rows = []
    
 
    # for row in df.iloc[:, index].tolist():
        
    #     cleaned = " ".join(clean_text(row).split()[:20]) #select 20 words
        
    #     if cleaned != "":
    #         cleaned_rows.append(cleaned)
    
    
    # return " ".join(cleaned_rows[:5]) #select rows


In [ ]:
""" def get_table(file_name):
    if file_name in cta_train_gt["table_name"].tolist():
        path = 'SOTAB CTA/Train/' + file_name
        split = "train"
    else:
        path = 'SOTAB CTA/Test/' + file_name
        split = "test"

    df = pd.read_json(path, compression='gzip', lines=True)
    print(df)
    labeled_idx = gt[split][file_name]
    df = df[list(labeled_idx.keys())]

    cleaned_rows = []

    col_labels = " || ".join(f"Column {i}" for i in range(1, len(df.columns) + 1))

    cleaned_rows.append(f"{col_labels}")
    cleaned_row = ''
    for _, row in df[:5].iterrows():
        for index, col in enumerate(df.columns):
            cleaned_row += clean_text(row[col]) + "||"
        cleaned_row += "\n" 
    return cleaned_row


#Example usage
file_name = 'Product_divas-club.de_September2020_CTA.json.gz'
result = get_table(file_name)
print(result) """


In [ ]:
""" def get_table(file_name):
    if file_name in cta_train_gt["table_name"].tolist():
        path = 'SOTAB CTA/Train/' + file_name
        split = "train"
    else:
        path = 'SOTAB CTA/Test/' + file_name
        split = "test"

    df = pd.read_json(path, compression='gzip', lines=True)
    labeled_idx = gt[split][file_name]
    df = df[list(labeled_idx.keys())]

    cleaned_rows = []

    col_labels = " || ".join(f"Column {i}" for i in range(1, len(df.columns) + 1))
   
    cleaned_rows.append(f"{col_labels}")
    cleaned_row = ''


    for _, row in df[:5].iterrows():
        for index, col in enumerate(df.columns):
            cleaned_row += clean_text(row[col]) + "||"
        cleaned_row += "\n"
    cleaned_rows.append(cleaned_row)

    return cleaned_rows


#Example usage
file_name = 'Product_divas-club.de_September2020_CTA.json.gz'
result = get_table(file_name)
print(result) """


In [12]:
def get_table(file_name):
    if file_name in cta_train_gt["table_name"].tolist():
        path = 'SOTAB CTA/Train/' + file_name
        split = "train"
    else:
        path = 'SOTAB CTA/Test/' + file_name
        split = "test"

    df = pd.read_json(path, compression='gzip', lines=True)
    labeled_idx = gt[split][file_name]
    df = df[list(labeled_idx.keys())]
    
    cleaned_rows = []

    col_labels = " || ".join(f"Column {i}" for i in range(1, len(df.columns) + 1))
   
    cleaned_row = ''

    for _, row in df[:5].iterrows():
        if all(value is None for value in row):
            continue 
        cleaned_row += " || ".join(clean_text(row[col]) if row[col] is not None else '' for col in df.columns) + "||\n"
    if not cleaned_row.strip():
        for _, row in df[5:10].iterrows():
            if all(value is None for value in row):
                continue
            cleaned_row += " || ".join(clean_text(row[col]) if row[col] is not None else '' for col in df.columns) + "||\n"
    if cleaned_row.strip():
         cleaned_rows.append(f"{col_labels}||\n{cleaned_row}")

    return cleaned_rows



In [13]:
test_examples = []
for table in gt['test']:
    col_str = get_table(table)
    test_examples.append([table, col_str, list(gt['test'][table].values()), table.split("_")[0]])

In [12]:
get_table('Event_blugrottonj.com_September2020_CTA.json.gz')

             4               5
0   Blu Grotto  (732) 571-7900
1         None            None
2         None            None
3   Blu Grotto  (732) 571-7900
4         None            None
5         None            None
6         None            None
7         None            None
8         None            None
9         None            None
10        None            None
11        None            None
12        None            None
13        None            None
14        None            None
15        None            None


['Column 1 || Column 2||\nBlu Grotto || (732) 571-7900||\nBlu Grotto || (732) 571-7900||\n']

In [13]:
get_table('Movie_321movies.org_September2020_CTA.json.gz')

         10        3   5                      0 
0      None   93 Min.   1   Nobody Walks in L.A.
1         R   80 Min.   5                     XX
2      None      None   0  A House Is Not a Home
3         R   89 Min.   1               The Hunt
4        PG   94 Min.   0      Love & Friendship
...     ...       ...  ..                    ...
1005   None   84 Min.   0           House of VHS
1006   None   90 Min.   0                 Plan Z
1007  PG-13   97 Min.   0                   Sold
1008   None      None   0            Ambarsariya
1009  PG-13  115 Min.   0                   Thor

[1010 rows x 4 columns]


['Column 1 || Column 2 || Column 3 || Column 4||\n || 93 Min. || 1 || Nobody Walks in L.A.||\nR || 80 Min. || 5 || XX||\n ||  || 0 || A House Is Not a Home||\nR || 89 Min. || 1 || The Hunt||\nPG || 94 Min. || 0 || Love & Friendship||\n']

In [14]:
get_table('LocalBusiness_basellive.ch_September2020_CTA.json.gz')

                                                    2
0                                                None
1                                                None
2                                                None
3                                                None
4                                                None
5      Th 12:00-18:30, Fr 12:00-18:30, Sa 11:00-17:00
6                                                None
7   Mo 09:00-19:00, Tu 09:00-19:00, We 09:00-19:00...
8                                                None
9                                                None
10  We 14:00-19:00, Th 14:00-19:00, Fr 14:00-19:00...
11  Mo 11:00-23:00, Tu 11:00-23:00, We 11:00-23:00...
12  Mo 09:00-19:00, Tu 09:00-19:00, We 09:00-19:00...
13                                               None
14  Mo 09:00-21:00, Tu 09:00-21:00, We 09:00-21:00...
15                                               None
16                                               None
17  Mo 09:00-18:30, Tu 09:00

['Column 1||\nTh 12:00-18:30, Fr 12:00-18:30, Sa 11:00-17:00||\nMo 09:00-19:00, Tu 09:00-19:00, We 09:00-19:00, Th 09:00-19:00, Fr 09:00-19:00, Sa 09:00-18:00||\n']

In [15]:
get_table('Product_babyartikelcheck.de_September2020_CTA.json.gz')

                                     4     6     8     10    11    7      9   \
0                           BY 88 Smart   GRM   CMT   CMT   CMT   6.6    8.9   
1                                  Tody   KGM   CMT   CMT   CMT  48.0  102.0   
2   Salsa 4 Air Diamond Special Edition   KGM   CMT   CMT   CMT  63.0  110.0   
3                               Resea S   KGM   CMT   CMT   CMT  58.0  106.5   
4                Pacific 4 Shop N Drive  None  None  None  None   NaN    NaN   
5                                Apollo   KGM   CMT   CMT   CMT  55.0  107.0   
6                              Solstice   KGM   CMT   CMT   CMT  66.0  103.0   
7                            BY 99 Dual   GRM   CMT   CMT   CMT   6.5    9.0   
8                             Summit X3   KGM   CMT   CMT   CMT  66.5  110.5   
9                              Brisk LX   KGM   CMT   CMT   CMT  53.0  103.0   
10                                BY 33   GRM   CMT   CMT   CMT   6.9    7.0   
11                         Avent SCD625 

['Column 1 || Column 2 || Column 3 || Column 4 || Column 5 || Column 6 || Column 7 || Column 8 || Column 9 || Column 10||\nBY 88 Smart || GRM || CMT || CMT || CMT || 6.6 || 8.9 || 5.8 || 90.0 || Beurer BY 88 Smart||\nTody || KGM || CMT || CMT || CMT || 48.0 || 102.0 || 86.0 || 7.5 || Qtus Tody||\nSalsa 4 Air Diamond Special Edition || KGM || CMT || CMT || CMT || 63.0 || 110.0 || 95.0 || 14.5 || ABC Design Salsa 4 Air Diamond Special Edition||\nResea S || KGM || CMT || CMT || CMT || 58.0 || 106.5 || 95.0 || 11.5 || Moon Resea S||\nPacific 4 Shop N Drive ||  ||  ||  ||  ||  ||  ||  ||  || Hauck Pacific 4 Shop N Drive||\n']

In [14]:
test_examples

[['Event_blugrottonj.com_September2020_CTA.json.gz',
  ['Column 1 || Column 2||\nBlu Grotto || (732) 571-7900||\nBlu Grotto || (732) 571-7900||\n'],
  ['Place/name', 'telephone'],
  'Event'],
 ['LocalBusiness_basellive.ch_September2020_CTA.json.gz',
  ['Column 1||\nTh 12:00-18:30, Fr 12:00-18:30, Sa 11:00-17:00||\nMo 09:00-19:00, Tu 09:00-19:00, We 09:00-19:00, Th 09:00-19:00, Fr 09:00-19:00, Sa 09:00-18:00||\n'],
  ['openingHours'],
  'LocalBusiness'],
 ['LocalBusiness_worldcement.com_September2020_CTA.json.gz',
  ['Column 1 || Column 2||\nUnited Kingdom || Independent Cement Consultants (ICC)||\nUnited Kingdom || JAMCEM Consulting||\nUnited Kingdom || Mirion Technologies (IST) Limited||\nGermany || KIMA Echtzeitsysteme GmbH||\nUnited Kingdom || Lubrication Engineers International Ltd||\n'],
  ['Country', 'LocalBusiness/name'],
  'LocalBusiness'],
 ['Movie_321movies.org_September2020_CTA.json.gz',
  ['Column 1 || Column 2 || Column 3 || Column 4||\n || 93 Min. || 1 || Nobody Walks in 

In [15]:
len(test_examples)

110

In [18]:
train_examples = []
for table in gt['train']:
    col_str = get_table(table)
    train_examples.append([table, col_str, list(gt['train'][table].values()), table.split("_")[0]])

In [20]:
len(train_examples)

44769

In [23]:
import pickle

In [24]:
file_name='SOTAB CTA/sotabv2-cta-train-table.pkl'
f = open(file_name,'wb')
pickle.dump(train_examples,f)
f.close()

In [25]:
file_name='SOTAB CTA/sotabv2-cta-test-table.pkl'
f = open(file_name,'wb')
pickle.dump(test_examples,f)
f.close()